In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
#using model.save_weights() to save the weights of the model in HDF5 format
# model.save_weights("/content/gdrive/My Drive/weights.h5")

Mounted at /content/gdrive


In [3]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.0 MB/s eta 0:00:00


In [4]:
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 6.4 MB/s eta 0:00:00


In [5]:
!pip install transformers --upgrade
!pip install accelerate --upgrade

In [6]:
from datasets import load_dataset

ds = load_dataset('argilla/databricks-dolly-15k-curated-en', split = 'train[:5000]')

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [7]:
ds = ds.train_test_split(test_size=0.2)

In [28]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [9]:
# def formatting_prompts_func(examples):
#     output_texts = []
#     instruction = examples["original-instruction"]
#     context = examples["original-context"]
#     response = examples["original-response"]
#     text = f'''
#             Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

#             ### Instruction:
#             {instruction}

#             ### Input:
#             {context}

#             ### Response:
#             {response}
#             '''
#     output_texts.append(text)

#     return output_texts

# # ds_test = formatting_prompts_func(ds[0])

In [10]:
ds = ds.flatten()

ds['train'][0]

{'id': '4749',
 'category': 'open_qa',
 'original-instruction': 'How do you make the color green out of other colors?',
 'original-context': '',
 'original-response': 'You can make green by mixing blue and yellow together',
 'new-instruction.user_id': [None],
 'new-instruction.value': ['How do you make the color green out of other colors?'],
 'new-instruction.status': ['submitted'],
 'new-context.user_id': [None],
 'new-context.value': [''],
 'new-context.status': ['submitted'],
 'new-response.user_id': [None],
 'new-response.value': ['You can make green by mixing blue and yellow together'],
 'new-response.status': ['submitted'],
 'external_id': None}

In [11]:
def preprocess_function(example):
    # Extracting information from the provided example
    questions = [q.strip() for q in example["original-instruction"]]
    contexts = example["original-context"]
    responses = example["original-response"]
    answer_starts = [0] * len(responses)  # Assuming the answer starts at the beginning of each response

    # Initialize empty lists for questions, contexts, and responses
    new_questions = []
    new_contexts = []
    new_responses = []

    # Iterate over examples and filter out short sequences
    for i, question in enumerate(questions):
        context = contexts[i]
        response = responses[i]

        # Check if the combined length of question and context is long enough
        if len(question) + len(context) > tokenizer.model_max_length - 4:  # 4 is for special tokens
            # Skip this example if it's too short
            continue

        # Append the examples that meet the length criteria
        new_questions.append(question)
        new_contexts.append(context)
        new_responses.append(response)

    # Tokenizing the input using the tokenizer
    inputs = tokenizer(
        new_questions,
        new_contexts,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
        max_length=tokenizer.model_max_length,
    )

    # Extracting the offset_mapping
    offset_mapping = inputs.pop("offset_mapping")

    # Initializing start and end positions
    start_positions = []
    end_positions = []

    # Calculating start and end positions for each response
    for i, offset in enumerate(offset_mapping):
        answer_start = answer_starts[i]
        response = new_responses[i]

        # Find the start and end token positions
        start_positions.append(answer_start)
        end_positions.append(answer_start + len(response))

    # Updating the inputs dictionary with start and end positions
    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions

    return inputs


In [31]:
tokenizer.pad_token = tokenizer.eos_token

In [32]:
dolly_ds_tokenized = ds.map(preprocess_function, batched=True, remove_columns=ds["train"].column_names)

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [13]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()

In [26]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

model = AutoModelForQuestionAnswering.from_pretrained("distilbert-base-uncased")

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

In [ ]:
training_args = TrainingArguments(
    output_dir="./colab_dolly_FT_gpt2",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dolly_ds_tokenized["train"],
    eval_dataset=dolly_ds_tokenized["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()

In [19]:
question = "How many programming languages does BLOOM support?"
context = "BLOOM has 176 billion parameters and can generate text in 46 languages natural languages and 13 programming languages."

# Load model directly
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

tokenizer = AutoTokenizer.from_pretrained("01GangaPutraBheeshma/colab_dolly_FT_gpt2")
model = AutoModelForQuestionAnswering.from_pretrained("01GangaPutraBheeshma/colab_dolly_FT_gpt2")


from transformers import pipeline

question_answerer = pipeline("question-answering", model=model, tokenizer = tokenizer)
question_answerer(question=question, context=context)

{'score': 2.924242380686337e-06,
 'start': 89,
 'end': 118,
 'answer': 'and 13 programming languages.'}